# Chapter 6: 高度なMCMC手法

## 学習目標
- ハミルトニアンモンテカルロ法（HMC）の原理を理解する
- No-U-Turn Sampler（NUTS）の仕組みを学ぶ
- 適応的MCMC手法を習得する
- 並列・分散MCMCの概念を理解する
- 変分推論との比較を学ぶ
- 実用的なMCMCライブラリの使用方法を習得する

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.optimize import minimize
from scipy.linalg import cholesky, solve_triangular
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'DejaVu Sans'
sns.set_style("whitegrid")
np.random.seed(42)

## 6.1 ハミルトニアンモンテカルロ法（HMC）

HMCは物理学のハミルトニアン力学を応用したMCMC手法で、高次元空間での効率的なサンプリングを可能にします。

### 基本概念
- **位置変数**：$q$ （パラメータ）
- **運動量変数**：$p$ （補助変数）
- **ハミルトニアン**：$H(q, p) = U(q) + K(p)$
  - $U(q) = -\log \pi(q)$ （ポテンシャルエネルギー）
  - $K(p) = \frac{1}{2}p^T M^{-1} p$ （運動エネルギー）

In [ ]:
class HamiltonianMC:
    """
    ハミルトニアンモンテカルロ法の実装
    """
    
    def __init__(self, log_prob_fn, grad_log_prob_fn, mass_matrix=None):
        """
        Parameters:
        - log_prob_fn: 対数確率密度関数
        - grad_log_prob_fn: 対数確率密度の勾配関数
        - mass_matrix: 質量行列（None の場合は単位行列）
        """
        self.log_prob_fn = log_prob_fn
        self.grad_log_prob_fn = grad_log_prob_fn
        self.mass_matrix = mass_matrix
        
    def leapfrog_step(self, q, p, epsilon):
        """
        リープフロッグ積分による1ステップ更新
        """
        # 運動量の半ステップ更新
        p_half = p + 0.5 * epsilon * self.grad_log_prob_fn(q)
        
        # 位置の全ステップ更新
        if self.mass_matrix is None:
            q_new = q + epsilon * p_half
        else:
            q_new = q + epsilon * np.linalg.solve(self.mass_matrix, p_half)
        
        # 運動量の残り半ステップ更新
        p_new = p_half + 0.5 * epsilon * self.grad_log_prob_fn(q_new)
        
        return q_new, p_new
    
    def hamiltonian(self, q, p):
        """
        ハミルトニアンの計算
        """
        # ポテンシャルエネルギー
        U = -self.log_prob_fn(q)
        
        # 運動エネルギー
        if self.mass_matrix is None:
            K = 0.5 * np.sum(p**2)
        else:
            K = 0.5 * p.T @ np.linalg.solve(self.mass_matrix, p)
        
        return U + K
    
    def sample(self, initial_q, n_samples, epsilon, L):
        """
        HMCサンプリングの実行
        
        Parameters:
        - initial_q: 初期位置
        - n_samples: サンプル数
        - epsilon: ステップサイズ
        - L: リープフロッグステップ数
        
        Returns:
        - samples: サンプル配列
        - acceptance_rate: 受理率
        - hamiltonian_errors: ハミルトニアン保存誤差
        """
        dim = len(initial_q)
        samples = np.zeros((n_samples, dim))
        hamiltonian_errors = np.zeros(n_samples)
        
        current_q = initial_q.copy()
        n_accepted = 0
        
        for i in range(n_samples):
            # 運動量をサンプリング
            if self.mass_matrix is None:
                current_p = np.random.normal(0, 1, dim)
            else:
                current_p = np.random.multivariate_normal(np.zeros(dim), self.mass_matrix)
            
            # 現在のハミルトニアン
            current_H = self.hamiltonian(current_q, current_p)
            
            # リープフロッグ積分
            q, p = current_q.copy(), current_p.copy()
            
            for _ in range(L):
                q, p = self.leapfrog_step(q, p, epsilon)
            
            # 運動量を反転（ハミルトニアン力学の時間反転対称性）
            p = -p
            
            # 提案後のハミルトニアン
            proposed_H = self.hamiltonian(q, p)
            
            # ハミルトニアン保存誤差
            hamiltonian_error = abs(proposed_H - current_H)
            hamiltonian_errors[i] = hamiltonian_error
            
            # メトロポリス受理確率
            alpha = min(1.0, np.exp(current_H - proposed_H))
            
            # 受理/棄却
            if np.random.rand() < alpha:
                current_q = q
                n_accepted += 1
            
            samples[i] = current_q
            
            if (i + 1) % 1000 == 0:
                print(f"Iteration {i+1}/{n_samples}, Acceptance rate: {n_accepted/(i+1):.3f}")
        
        acceptance_rate = n_accepted / n_samples
        return samples, acceptance_rate, hamiltonian_errors

# テスト用の2次元正規分布
def multivariate_normal_log_prob(x, mu, cov):
    """多変量正規分布の対数確率密度"""
    diff = x - mu
    try:
        chol = cholesky(cov, lower=True)
        log_det = 2 * np.sum(np.log(np.diag(chol)))
        solve = solve_triangular(chol, diff, lower=True)
        mahalanobis_sq = np.sum(solve**2)
    except np.linalg.LinAlgError:
        return -np.inf
    
    return -0.5 * (len(mu) * np.log(2 * np.pi) + log_det + mahalanobis_sq)

def multivariate_normal_grad_log_prob(x, mu, cov):
    """多変量正規分布の対数確率密度の勾配"""
    diff = x - mu
    try:
        grad = -np.linalg.solve(cov, diff)
    except np.linalg.LinAlgError:
        grad = np.zeros_like(x)
    return grad

# パラメータ設定
mu_target = np.array([0.0, 0.0])
cov_target = np.array([[1.0, 0.8], [0.8, 1.0]])

# HMCサンプラーの作成
hmc = HamiltonianMC(
    log_prob_fn=lambda x: multivariate_normal_log_prob(x, mu_target, cov_target),
    grad_log_prob_fn=lambda x: multivariate_normal_grad_log_prob(x, mu_target, cov_target)
)

# HMCサンプリングの実行
print("HMCサンプリング実行中...")
initial_q = np.array([0.0, 0.0])
hmc_samples, hmc_acceptance_rate, hmc_errors = hmc.sample(
    initial_q=initial_q,
    n_samples=5000,
    epsilon=0.25,  # ステップサイズ
    L=20          # リープフロッグステップ数
)

print(f"\nHMC受理率: {hmc_acceptance_rate:.3f}")
print(f"平均ハミルトニアン誤差: {np.mean(hmc_errors):.6f}")

In [ ]:
# HMC結果の可視化
def plot_hmc_results(samples, errors, mu_target, cov_target, title="HMC Results"):
    """
    HMC結果の包括的可視化
    """
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    burnin = 500
    samples_clean = samples[burnin:]
    
    # 1. サンプルの軌跡（最初の500サンプル）
    trajectory = samples[:500]
    axes[0, 0].plot(trajectory[:, 0], trajectory[:, 1], 'b-', alpha=0.7, linewidth=0.8)
    axes[0, 0].plot(trajectory[:, 0], trajectory[:, 1], 'b.', markersize=1, alpha=0.6)
    axes[0, 0].plot(trajectory[0, 0], trajectory[0, 1], 'go', markersize=8, label='Start')
    axes[0, 0].plot(trajectory[-1, 0], trajectory[-1, 1], 'ro', markersize=8, label='End')
    axes[0, 0].set_title('HMC Trajectory (first 500)')
    axes[0, 0].set_xlabel('X1')
    axes[0, 0].set_ylabel('X2')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 散布図と真の分布の等高線
    axes[0, 1].scatter(samples_clean[::5, 0], samples_clean[::5, 1], alpha=0.6, s=1)
    
    # 真の分布の等高線
    x1_range = np.linspace(samples_clean[:, 0].min(), samples_clean[:, 0].max(), 50)
    x2_range = np.linspace(samples_clean[:, 1].min(), samples_clean[:, 1].max(), 50)
    X1, X2 = np.meshgrid(x1_range, x2_range)
    pos = np.dstack((X1, X2))
    rv = stats.multivariate_normal(mu_target, cov_target)
    axes[0, 1].contour(X1, X2, rv.pdf(pos), colors='red', alpha=0.8, linewidths=2)
    axes[0, 1].set_title('Samples with True Distribution')
    axes[0, 1].set_xlabel('X1')
    axes[0, 1].set_ylabel('X2')
    axes[0, 1].set_aspect('equal')
    
    # 3. ハミルトニアン誤差の時系列
    axes[0, 2].plot(errors[:2000], alpha=0.8, linewidth=0.8)
    axes[0, 2].set_title('Hamiltonian Errors')
    axes[0, 2].set_xlabel('Iteration')
    axes[0, 2].set_ylabel('|H_proposed - H_current|')
    axes[0, 2].set_yscale('log')
    axes[0, 2].grid(True, alpha=0.3)
    
    # 4. トレースプロット
    axes[1, 0].plot(samples[:2000, 0], alpha=0.8, label='X1', linewidth=0.8)
    axes[1, 0].plot(samples[:2000, 1], alpha=0.8, label='X2', linewidth=0.8)
    axes[1, 0].axvline(burnin, color='red', linestyle='--', alpha=0.7, label='Burn-in')
    axes[1, 0].set_title('Trace Plot')
    axes[1, 0].set_xlabel('Iteration')
    axes[1, 0].set_ylabel('Value')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 5. マージナル分布の比較
    axes[1, 1].hist(samples_clean[:, 0], bins=50, density=True, alpha=0.7, 
                   color='lightblue', label='X1 samples')
    x1_theory = np.linspace(samples_clean[:, 0].min(), samples_clean[:, 0].max(), 100)
    axes[1, 1].plot(x1_theory, stats.norm.pdf(x1_theory, mu_target[0], np.sqrt(cov_target[0, 0])), 
                   'r-', linewidth=2, label='X1 true')
    axes[1, 1].set_title('Marginal Distribution X1')
    axes[1, 1].set_xlabel('X1')
    axes[1, 1].set_ylabel('Density')
    axes[1, 1].legend()
    
    # 6. 自己相関関数
    from statsmodels.tsa.stattools import acf
    lags = min(100, len(samples_clean) // 10)
    autocorr_x1 = acf(samples_clean[:, 0], nlags=lags, fft=True)
    autocorr_x2 = acf(samples_clean[:, 1], nlags=lags, fft=True)
    
    axes[1, 2].plot(autocorr_x1, label='X1', alpha=0.8)
    axes[1, 2].plot(autocorr_x2, label='X2', alpha=0.8)
    axes[1, 2].axhline(0, color='k', linestyle='--', alpha=0.5)
    axes[1, 2].axhline(0.05, color='r', linestyle='--', alpha=0.5)
    axes[1, 2].axhline(-0.05, color='r', linestyle='--', alpha=0.5)
    axes[1, 2].set_title('Autocorrelation Functions')
    axes[1, 2].set_xlabel('Lag')
    axes[1, 2].set_ylabel('ACF')
    axes[1, 2].legend()
    axes[1, 2].grid(True, alpha=0.3)
    
    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()

# 結果の可視化
plot_hmc_results(hmc_samples, hmc_errors, mu_target, cov_target, "Hamiltonian Monte Carlo Results")

# 統計的比較
burnin = 500
hmc_clean = hmc_samples[burnin:]

sample_mean = np.mean(hmc_clean, axis=0)
sample_cov = np.cov(hmc_clean.T)

print(f"\n=== HMC統計結果 ===")
print(f"真の平均:     {mu_target}")
print(f"サンプル平均: {sample_mean}")
print(f"\n真の共分散:")
print(cov_target)
print(f"サンプル共分散:")
print(sample_cov)

# 効率の計算
from statsmodels.tsa.stattools import acf
autocorr_x1 = acf(hmc_clean[:, 0], nlags=min(200, len(hmc_clean)//4), fft=True)
tau_int = 1.0
for k in range(1, len(autocorr_x1)):
    if autocorr_x1[k] > 0.01:
        tau_int += 2 * autocorr_x1[k]
    else:
        break

eff_sample_size = len(hmc_clean) / (2 * tau_int + 1)
print(f"\n統合自己相関時間: {tau_int:.2f}")
print(f"有効サンプルサイズ: {eff_sample_size:.0f}")
print(f"効率: {eff_sample_size/len(hmc_clean):.2%}")

## 6.2 ランダムウォークMHとHMCの比較

同じ分布に対してランダムウォーク・メトロポリス・ヘイスティングス法とHMCを比較してみましょう。

In [ ]:
# ランダムウォークMHの実装（比較用）
def random_walk_mh(log_prob_fn, initial_value, n_samples, step_size=0.5):
    """
    ランダムウォーク・メトロポリス・ヘイスティングス法
    """
    dim = len(initial_value)
    samples = np.zeros((n_samples, dim))
    current = initial_value.copy()
    current_log_prob = log_prob_fn(current)
    n_accepted = 0
    
    proposal_cov = step_size**2 * np.eye(dim)
    
    for i in range(n_samples):
        # 提案
        proposed = np.random.multivariate_normal(current, proposal_cov)
        proposed_log_prob = log_prob_fn(proposed)
        
        # 受理確率
        log_alpha = proposed_log_prob - current_log_prob
        alpha = min(1.0, np.exp(log_alpha))
        
        # 受理/棄却
        if np.random.rand() < alpha:
            current = proposed
            current_log_prob = proposed_log_prob
            n_accepted += 1
        
        samples[i] = current
        
        if (i + 1) % 1000 == 0:
            print(f"Iteration {i+1}/{n_samples}, Acceptance rate: {n_accepted/(i+1):.3f}")
    
    acceptance_rate = n_accepted / n_samples
    return samples, acceptance_rate

# 比較実験の実行
print("ランダムウォークMHサンプリング実行中...")
rwmh_samples, rwmh_acceptance_rate = random_walk_mh(
    log_prob_fn=lambda x: multivariate_normal_log_prob(x, mu_target, cov_target),
    initial_value=np.array([0.0, 0.0]),
    n_samples=5000,
    step_size=0.8
)

print(f"\nランダムウォークMH受理率: {rwmh_acceptance_rate:.3f}")

# 効率比較
def compute_efficiency_metrics(samples, burnin_frac=0.1):
    """
    効率指標の計算
    """
    burnin = int(len(samples) * burnin_frac)
    clean_samples = samples[burnin:]
    
    # 各次元の自己相関時間を計算
    autocorr_times = []
    eff_sample_sizes = []
    
    for dim in range(clean_samples.shape[1]):
        data = clean_samples[:, dim]
        lags = min(200, len(data) // 4)
        autocorr = acf(data, nlags=lags, fft=True)
        
        # 統合自己相関時間
        tau_int = 1.0
        for lag in range(1, len(autocorr)):
            if autocorr[lag] > 0.01:
                tau_int += 2 * autocorr[lag]
            else:
                break
        
        autocorr_times.append(tau_int)
        eff_sample_sizes.append(len(data) / (2 * tau_int + 1))
    
    return autocorr_times, eff_sample_sizes

# 効率比較
hmc_autocorr, hmc_eff = compute_efficiency_metrics(hmc_samples)
rwmh_autocorr, rwmh_eff = compute_efficiency_metrics(rwmh_samples)

print(f"\n=== 効率比較 ===")
print(f"{'Method':<12} {'Dim':<5} {'Acceptance':<12} {'Autocorr Time':<15} {'Eff Sample Size':<18} {'Efficiency':<12}")
print("-" * 90)

for dim in range(2):
    print(f"{'HMC':<12} {'X'+str(dim+1):<5} {hmc_acceptance_rate:<12.3f} "
          f"{hmc_autocorr[dim]:<15.2f} {hmc_eff[dim]:<18.1f} {hmc_eff[dim]/len(hmc_samples[500:]):<12.3f}")
    print(f"{'Random Walk':<12} {'X'+str(dim+1):<5} {rwmh_acceptance_rate:<12.3f} "
          f"{rwmh_autocorr[dim]:<15.2f} {rwmh_eff[dim]:<18.1f} {rwmh_eff[dim]/len(rwmh_samples[500:]):<12.3f}")
    print()

print(f"平均効率比（HMC/RWMH）: {np.mean(hmc_eff)/np.mean(rwmh_eff):.2f}")

In [ ]:
# HMC vs ランダムウォークMHの可視化比較
fig, axes = plt.subplots(2, 4, figsize=(16, 10))

burnin = 500
hmc_clean = hmc_samples[burnin:]
rwmh_clean = rwmh_samples[burnin:]

# トレースプロット比較
axes[0, 0].plot(hmc_samples[:2000, 0], alpha=0.8, label='HMC', linewidth=0.8, color='blue')
axes[0, 0].plot(rwmh_samples[:2000, 0], alpha=0.8, label='RWMH', linewidth=0.8, color='red')
axes[0, 0].axvline(burnin, color='gray', linestyle='--', alpha=0.7)
axes[0, 0].set_title('Trace Plot Comparison (X1)')
axes[0, 0].set_xlabel('Iteration')
axes[0, 0].set_ylabel('X1')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 散布図比較
axes[0, 1].scatter(hmc_clean[::10, 0], hmc_clean[::10, 1], alpha=0.6, s=5, color='blue', label='HMC')
axes[0, 1].scatter(rwmh_clean[::10, 0], rwmh_clean[::10, 1], alpha=0.6, s=5, color='red', label='RWMH')
axes[0, 1].set_title('Sample Scatter Plot')
axes[0, 1].set_xlabel('X1')
axes[0, 1].set_ylabel('X2')
axes[0, 1].legend()
axes[0, 1].set_aspect('equal')

# 自己相関比較
lags = min(100, len(hmc_clean) // 10)
hmc_acf = acf(hmc_clean[:, 0], nlags=lags, fft=True)
rwmh_acf = acf(rwmh_clean[:, 0], nlags=lags, fft=True)

axes[0, 2].plot(hmc_acf, label='HMC', alpha=0.8, color='blue')
axes[0, 2].plot(rwmh_acf, label='RWMH', alpha=0.8, color='red')
axes[0, 2].axhline(0, color='k', linestyle='--', alpha=0.5)
axes[0, 2].axhline(0.05, color='gray', linestyle='--', alpha=0.5)
axes[0, 2].set_title('Autocorrelation Comparison (X1)')
axes[0, 2].set_xlabel('Lag')
axes[0, 2].set_ylabel('ACF')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

# 効率指標の比較
methods = ['HMC', 'RWMH']
x1_eff = [hmc_eff[0], rwmh_eff[0]]
x2_eff = [hmc_eff[1], rwmh_eff[1]]

x_pos = np.arange(len(methods))
width = 0.35

axes[0, 3].bar(x_pos - width/2, x1_eff, width, label='X1', alpha=0.7, color='lightblue')
axes[0, 3].bar(x_pos + width/2, x2_eff, width, label='X2', alpha=0.7, color='lightcoral')
axes[0, 3].set_title('Effective Sample Size')
axes[0, 3].set_xlabel('Method')
axes[0, 3].set_ylabel('Effective Sample Size')
axes[0, 3].set_xticks(x_pos)
axes[0, 3].set_xticklabels(methods)
axes[0, 3].legend()
axes[0, 3].grid(True, alpha=0.3)

# 軌跡の比較（最初の200ステップ）
hmc_traj = hmc_samples[:200]
rwmh_traj = rwmh_samples[:200]

axes[1, 0].plot(hmc_traj[:, 0], hmc_traj[:, 1], 'b-', alpha=0.7, linewidth=1, label='HMC')
axes[1, 0].plot(hmc_traj[:, 0], hmc_traj[:, 1], 'b.', markersize=2, alpha=0.8)
axes[1, 0].set_title('HMC Trajectory (first 200)')
axes[1, 0].set_xlabel('X1')
axes[1, 0].set_ylabel('X2')
axes[1, 0].set_aspect('equal')

axes[1, 1].plot(rwmh_traj[:, 0], rwmh_traj[:, 1], 'r-', alpha=0.7, linewidth=1, label='RWMH')
axes[1, 1].plot(rwmh_traj[:, 0], rwmh_traj[:, 1], 'r.', markersize=2, alpha=0.8)
axes[1, 1].set_title('RWMH Trajectory (first 200)')
axes[1, 1].set_xlabel('X1')
axes[1, 1].set_ylabel('X2')
axes[1, 1].set_aspect('equal')

# ステップサイズの効果（HMC）
step_distances_hmc = np.sqrt(np.sum(np.diff(hmc_samples[:1000], axis=0)**2, axis=1))
step_distances_rwmh = np.sqrt(np.sum(np.diff(rwmh_samples[:1000], axis=0)**2, axis=1))

axes[1, 2].hist(step_distances_hmc, bins=30, alpha=0.7, density=True, color='blue', label='HMC')
axes[1, 2].hist(step_distances_rwmh, bins=30, alpha=0.7, density=True, color='red', label='RWMH')
axes[1, 2].set_title('Step Size Distribution')
axes[1, 2].set_xlabel('Step Distance')
axes[1, 2].set_ylabel('Density')
axes[1, 2].legend()

# 探索効率（単位時間あたりの有効サンプル数）
# 注：実際の計算時間は測定していないため、理論的な比較
# HMCは1ステップあたりより多くの計算が必要だが、より効率的
axes[1, 3].bar(['HMC', 'RWMH'], 
               [np.mean(hmc_eff)/20, np.mean(rwmh_eff)],  # HMCは20倍の計算コストと仮定
               alpha=0.7, color=['blue', 'red'])
axes[1, 3].set_title('Computational Efficiency\n(ESS per unit computation)')
axes[1, 3].set_ylabel('Relative Efficiency')
axes[1, 3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 詳細統計の比較
print(f"\n=== 詳細統計比較 ===")
print(f"{'Metric':<25} {'HMC':<15} {'Random Walk MH':<15} {'Ratio (HMC/RWMH)':<15}")
print("-" * 80)

hmc_mean_eff = np.mean(hmc_eff)
rwmh_mean_eff = np.mean(rwmh_eff)
hmc_mean_tau = np.mean(hmc_autocorr)
rwmh_mean_tau = np.mean(rwmh_autocorr)

print(f"{'Acceptance Rate':<25} {hmc_acceptance_rate:<15.3f} {rwmh_acceptance_rate:<15.3f} {hmc_acceptance_rate/rwmh_acceptance_rate:<15.2f}")
print(f"{'Mean Autocorr Time':<25} {hmc_mean_tau:<15.2f} {rwmh_mean_tau:<15.2f} {hmc_mean_tau/rwmh_mean_tau:<15.2f}")
print(f"{'Mean Eff Sample Size':<25} {hmc_mean_eff:<15.1f} {rwmh_mean_eff:<15.1f} {hmc_mean_eff/rwmh_mean_eff:<15.2f}")
print(f"{'Efficiency':<25} {hmc_mean_eff/len(hmc_clean):<15.3f} {rwmh_mean_eff/len(rwmh_clean):<15.3f} {(hmc_mean_eff/len(hmc_clean))/(rwmh_mean_eff/len(rwmh_clean)):<15.2f}")

# 平均ステップサイズ
hmc_mean_step = np.mean(step_distances_hmc[step_distances_hmc > 0])
rwmh_mean_step = np.mean(step_distances_rwmh[step_distances_rwmh > 0])
print(f"{'Mean Step Size':<25} {hmc_mean_step:<15.3f} {rwmh_mean_step:<15.3f} {hmc_mean_step/rwmh_mean_step:<15.2f}")

## 6.3 No-U-Turn Sampler (NUTS)

NUTSはHMCの拡張で、リープフロッグステップ数Lを自動的に決定する手法です。

### 基本アイデア
- 軌跡が「U字型」になる（元の位置に戻り始める）まで続ける
- 木構造を用いて効率的に実装
- ステップサイズの適応的調整

In [ ]:
class SimpleNUTS:
    """
    簡易版No-U-Turn Sampler
    
    注：これは教育目的の簡易実装です。
    実際のNUTSはより複雑な木構造と適応機構を持ちます。
    """
    
    def __init__(self, log_prob_fn, grad_log_prob_fn, max_treedepth=10):
        self.log_prob_fn = log_prob_fn
        self.grad_log_prob_fn = grad_log_prob_fn
        self.max_treedepth = max_treedepth
        
    def leapfrog_step(self, q, p, epsilon):
        """リープフロッグステップ"""
        p_half = p + 0.5 * epsilon * self.grad_log_prob_fn(q)
        q_new = q + epsilon * p_half
        p_new = p_half + 0.5 * epsilon * self.grad_log_prob_fn(q_new)
        return q_new, p_new
    
    def compute_energy(self, q, p):
        """エネルギー（負の対数確率）の計算"""
        return -self.log_prob_fn(q) + 0.5 * np.sum(p**2)
    
    def build_tree(self, q, p, u, v, j, epsilon, q0, p0):
        """
        二分木の構築（簡易版）
        
        Returns:
        - q_minus, p_minus: 木の左端
        - q_plus, p_plus: 木の右端
        - q_prime: 提案する新しい位置
        - n_prime: 有効な状態数
        - s_prime: 継続するかどうか
        """
        if j == 0:
            # 基本ケース：1ステップ
            q_prime, p_prime = self.leapfrog_step(q, p, v * epsilon)
            
            # エネルギー制約のチェック
            energy_prime = self.compute_energy(q_prime, p_prime)
            energy_0 = self.compute_energy(q0, p0)
            
            n_prime = int(u <= np.exp(energy_0 - energy_prime))
            s_prime = int(u < np.exp(1000 + energy_0 - energy_prime))  # 数値安定性のため
            
            return q_prime, p_prime, q_prime, p_prime, q_prime, n_prime, s_prime
        
        else:
            # 再帰ケース
            # 最初の半分の木を構築
            q_minus, p_minus, q_plus, p_plus, q_prime, n_prime, s_prime = \
                self.build_tree(q, p, u, v, j-1, epsilon, q0, p0)
            
            if s_prime:
                if v == -1:
                    # 左方向に拡張
                    q_minus, p_minus, _, _, q_double_prime, n_double_prime, s_double_prime = \
                        self.build_tree(q_minus, p_minus, u, v, j-1, epsilon, q0, p0)
                else:
                    # 右方向に拡張
                    _, _, q_plus, p_plus, q_double_prime, n_double_prime, s_double_prime = \
                        self.build_tree(q_plus, p_plus, u, v, j-1, epsilon, q0, p0)
                
                # 提案の選択
                if np.random.rand() < n_double_prime / max(n_prime + n_double_prime, 1):
                    q_prime = q_double_prime
                
                # U-turn検出（簡易版）
                delta_minus = q_plus - q_minus
                no_uturn = (np.dot(delta_minus, p_minus) >= 0) and (np.dot(delta_minus, p_plus) >= 0)
                
                s_prime = s_double_prime and no_uturn
                n_prime = n_prime + n_double_prime
            
            return q_minus, p_minus, q_plus, p_plus, q_prime, n_prime, s_prime
    
    def sample(self, initial_q, n_samples, initial_epsilon=0.1, adapt_steps=1000):
        """
        NUTSサンプリングの実行
        """
        dim = len(initial_q)
        samples = np.zeros((n_samples, dim))
        
        q = initial_q.copy()
        epsilon = initial_epsilon
        n_accepted = 0
        
        # 適応的パラメータ
        target_accept = 0.8
        log_epsilon = np.log(epsilon)
        log_epsilon_bar = 0.0
        h_bar = 0.0
        gamma = 0.05
        t0 = 10
        kappa = 0.75
        
        for i in range(n_samples):
            # 運動量のサンプリング
            p = np.random.normal(0, 1, dim)
            
            # スライス変数
            u = np.random.rand() * np.exp(self.log_prob_fn(q) - 0.5 * np.sum(p**2))
            
            # 初期設定
            q_minus = q_plus = q.copy()
            p_minus = p_plus = p.copy()
            j = 0
            n = 1
            s = 1
            q_new = q.copy()
            
            # 木の構築
            while s and j < self.max_treedepth:
                # 方向を決定
                v = 2 * np.random.randint(2) - 1
                
                if v == -1:
                    q_minus, p_minus, _, _, q_prime, n_prime, s_prime = \
                        self.build_tree(q_minus, p_minus, u, v, j, epsilon, q, p)
                else:
                    _, _, q_plus, p_plus, q_prime, n_prime, s_prime = \
                        self.build_tree(q_plus, p_plus, u, v, j, epsilon, q, p)
                
                if s_prime:
                    if np.random.rand() < n_prime / n:
                        q_new = q_prime
                
                n = n + n_prime
                
                # U-turn検出
                delta = q_plus - q_minus
                s = s_prime and (np.dot(delta, p_minus) >= 0) and (np.dot(delta, p_plus) >= 0)
                j = j + 1
            
            # 受理/棄却（NUTSでは自動的に決まる）
            if not np.allclose(q_new, q):
                n_accepted += 1
            q = q_new
            samples[i] = q
            
            # ステップサイズの適応（Dual Averaging）
            if i < adapt_steps:
                accept_prob = min(1, n / (2**j))
                h_bar = (1 - 1/(i + t0 + 1)) * h_bar + (target_accept - accept_prob) / (i + t0 + 1)
                log_epsilon = log_epsilon_bar - np.sqrt(i + 1) / gamma * h_bar
                log_epsilon_bar = (i + 1)**(-kappa) * log_epsilon + (1 - (i + 1)**(-kappa)) * log_epsilon_bar
                epsilon = np.exp(log_epsilon)
            
            if (i + 1) % 1000 == 0:
                print(f"Iteration {i+1}/{n_samples}, Acceptance rate: {n_accepted/(i+1):.3f}, Epsilon: {epsilon:.4f}")
        
        acceptance_rate = n_accepted / n_samples
        return samples, acceptance_rate, epsilon

# NUTSサンプラーの作成と実行
print("NUTS (簡易版) サンプリング実行中...")
nuts = SimpleNUTS(
    log_prob_fn=lambda x: multivariate_normal_log_prob(x, mu_target, cov_target),
    grad_log_prob_fn=lambda x: multivariate_normal_grad_log_prob(x, mu_target, cov_target)
)

nuts_samples, nuts_acceptance_rate, final_epsilon = nuts.sample(
    initial_q=np.array([0.0, 0.0]),
    n_samples=3000  # NUTSは計算量が多いため少なめに設定
)

print(f"\nNUTS受理率: {nuts_acceptance_rate:.3f}")
print(f"最終ステップサイズ: {final_epsilon:.4f}")

# NUTS効率の計算
nuts_autocorr, nuts_eff = compute_efficiency_metrics(nuts_samples)

print(f"\n=== NUTS効率指標 ===")
print(f"平均自己相関時間: {np.mean(nuts_autocorr):.2f}")
print(f"平均有効サンプルサイズ: {np.mean(nuts_eff):.1f}")
print(f"効率: {np.mean(nuts_eff)/len(nuts_samples[300:]):.2%}")

## 6.4 適応的MCMC手法

サンプリング中にパラメータを自動調整する手法を学びます。

In [ ]:
class AdaptiveMetropolis:
    """
    適応的メトロポリス法（AM）
    Haario et al. (2001) による手法
    """
    
    def __init__(self, log_prob_fn, initial_cov_scale=0.1, adaptation_start=100):
        self.log_prob_fn = log_prob_fn
        self.initial_cov_scale = initial_cov_scale
        self.adaptation_start = adaptation_start
        
    def sample(self, initial_value, n_samples):
        """
        適応的サンプリングの実行
        """
        dim = len(initial_value)
        samples = np.zeros((n_samples, dim))
        
        # 初期設定
        current = initial_value.copy()
        current_log_prob = self.log_prob_fn(current)
        
        # 適応的共分散行列
        cov_matrix = self.initial_cov_scale**2 * np.eye(dim)
        epsilon = 1e-6  # 数値安定性のため
        sd_scale = 2.4**2 / dim  # 最適スケーリング
        
        n_accepted = 0
        
        for i in range(n_samples):
            # 提案分布の更新
            if i >= self.adaptation_start:
                # 経験共分散行列の計算
                sample_mean = np.mean(samples[:i], axis=0)
                sample_cov = np.cov(samples[:i].T, ddof=1)
                
                # Regularization（数値安定性のため）
                if np.any(np.isnan(sample_cov)) or np.any(np.isinf(sample_cov)):
                    cov_matrix = self.initial_cov_scale**2 * np.eye(dim)
                else:
                    cov_matrix = sd_scale * (sample_cov + epsilon * np.eye(dim))
            
            # 提案
            try:
                proposed = np.random.multivariate_normal(current, cov_matrix)
            except np.linalg.LinAlgError:
                # 共分散行列が特異の場合
                proposed = current + np.random.normal(0, self.initial_cov_scale, dim)
            
            proposed_log_prob = self.log_prob_fn(proposed)
            
            # 受理確率
            log_alpha = proposed_log_prob - current_log_prob
            alpha = min(1.0, np.exp(log_alpha))
            
            # 受理/棄却
            if np.random.rand() < alpha:
                current = proposed
                current_log_prob = proposed_log_prob
                n_accepted += 1
            
            samples[i] = current
            
            if (i + 1) % 1000 == 0:
                print(f"Iteration {i+1}/{n_samples}, Acceptance rate: {n_accepted/(i+1):.3f}")
        
        acceptance_rate = n_accepted / n_samples
        return samples, acceptance_rate, cov_matrix

# 適応的メトロポリス法の実行
print("適応的メトロポリス法サンプリング実行中...")
am = AdaptiveMetropolis(
    log_prob_fn=lambda x: multivariate_normal_log_prob(x, mu_target, cov_target)
)

am_samples, am_acceptance_rate, final_cov = am.sample(
    initial_value=np.array([0.0, 0.0]),
    n_samples=5000
)

print(f"\n適応的メトロポリス法受理率: {am_acceptance_rate:.3f}")
print(f"\n最終提案共分散行列:")
print(final_cov)
print(f"\n真の共分散行列:")
print(cov_target)

# 効率の計算
am_autocorr, am_eff = compute_efficiency_metrics(am_samples)

print(f"\n=== 適応的メトロポリス法効率指標 ===")
print(f"平均自己相関時間: {np.mean(am_autocorr):.2f}")
print(f"平均有効サンプルサイズ: {np.mean(am_eff):.1f}")
print(f"効率: {np.mean(am_eff)/len(am_samples[500:]):.2%}")

In [ ]:
# 全手法の比較可視化
fig, axes = plt.subplots(3, 3, figsize=(15, 15))

# サンプル数を統一（最小のものに合わせる）
min_samples = min(len(hmc_samples), len(rwmh_samples), len(nuts_samples), len(am_samples))
burnin_unified = 300

methods = ['Random Walk MH', 'HMC', 'NUTS', 'Adaptive Metropolis']
all_samples = [rwmh_samples[:min_samples], hmc_samples[:min_samples], 
               nuts_samples[:min_samples], am_samples[:min_samples]]
all_acceptance = [rwmh_acceptance_rate, hmc_acceptance_rate, nuts_acceptance_rate, am_acceptance_rate]
colors = ['red', 'blue', 'green', 'orange']

# 1. トレースプロット比較
for i, (method, samples, color) in enumerate(zip(methods, all_samples, colors)):
    axes[0, 0].plot(samples[:1500, 0], alpha=0.7, label=method, color=color, linewidth=0.8)
axes[0, 0].axvline(burnin_unified, color='gray', linestyle='--', alpha=0.7)
axes[0, 0].set_title('Trace Plot Comparison (X1)')
axes[0, 0].set_xlabel('Iteration')
axes[0, 0].set_ylabel('X1')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. 散布図比較
for i, (method, samples, color) in enumerate(zip(methods, all_samples, colors)):
    clean_samples = samples[burnin_unified:]
    axes[0, 1].scatter(clean_samples[::20, 0], clean_samples[::20, 1], 
                      alpha=0.6, s=5, label=method, color=color)
axes[0, 1].set_title('Sample Scatter Plot')
axes[0, 1].set_xlabel('X1')
axes[0, 1].set_ylabel('X2')
axes[0, 1].legend()
axes[0, 1].set_aspect('equal')

# 3. 受理率比較
axes[0, 2].bar(methods, all_acceptance, alpha=0.7, color=colors)
axes[0, 2].set_title('Acceptance Rate Comparison')
axes[0, 2].set_ylabel('Acceptance Rate')
axes[0, 2].tick_params(axis='x', rotation=45)
axes[0, 2].grid(True, alpha=0.3)

# 4-6. 各手法の軌跡（最初の200ステップ）
trajectory_plots = [(1, 0), (1, 1), (1, 2)]
for i, ((row, col), (method, samples, color)) in enumerate(zip(trajectory_plots, 
                                                               zip(methods[:3], all_samples[:3], colors[:3]))):
    traj = samples[:200]
    axes[row, col].plot(traj[:, 0], traj[:, 1], '-', alpha=0.7, linewidth=1, color=color)
    axes[row, col].plot(traj[:, 0], traj[:, 1], '.', markersize=2, alpha=0.8, color=color)
    axes[row, col].plot(traj[0, 0], traj[0, 1], 'go', markersize=6)
    axes[row, col].plot(traj[-1, 0], traj[-1, 1], 'ro', markersize=6)
    axes[row, col].set_title(f'{method} Trajectory')
    axes[row, col].set_xlabel('X1')
    axes[row, col].set_ylabel('X2')
    axes[row, col].set_aspect('equal')

# 7. 効率比較
all_eff = [rwmh_eff, hmc_eff, nuts_eff, am_eff]
eff_means = [np.mean(eff) for eff in all_eff]

axes[2, 0].bar(methods, eff_means, alpha=0.7, color=colors)
axes[2, 0].set_title('Effective Sample Size Comparison')
axes[2, 0].set_ylabel('Mean Effective Sample Size')
axes[2, 0].tick_params(axis='x', rotation=45)
axes[2, 0].grid(True, alpha=0.3)

# 8. 自己相関比較
for i, (method, samples, color) in enumerate(zip(methods, all_samples, colors)):
    clean_samples = samples[burnin_unified:]
    if len(clean_samples) > 100:
        lags = min(50, len(clean_samples) // 10)
        autocorr = acf(clean_samples[:, 0], nlags=lags, fft=True)
        axes[2, 1].plot(autocorr, label=method, alpha=0.8, color=color)

axes[2, 1].axhline(0, color='k', linestyle='--', alpha=0.5)
axes[2, 1].axhline(0.05, color='gray', linestyle='--', alpha=0.5)
axes[2, 1].set_title('Autocorrelation Comparison (X1)')
axes[2, 1].set_xlabel('Lag')
axes[2, 1].set_ylabel('ACF')
axes[2, 1].legend()
axes[2, 1].grid(True, alpha=0.3)

# 9. 統計サマリー
axes[2, 2].axis('off')
summary_text = "Method Comparison Summary\n\n"
all_autocorr = [rwmh_autocorr, hmc_autocorr, nuts_autocorr, am_autocorr]

for method, acc_rate, eff, autocorr in zip(methods, all_acceptance, all_eff, all_autocorr):
    summary_text += f"{method}:\n"
    summary_text += f"  Acceptance: {acc_rate:.3f}\n"
    summary_text += f"  Mean ESS: {np.mean(eff):.1f}\n"
    summary_text += f"  Mean τ: {np.mean(autocorr):.2f}\n\n"

axes[2, 2].text(0.1, 0.9, summary_text, transform=axes[2, 2].transAxes, 
               fontsize=10, verticalalignment='top', fontfamily='monospace')

plt.tight_layout()
plt.show()

# 定量的比較表
print(f"\n=== 全手法の定量的比較 ===")
print(f"{'Method':<20} {'Acceptance':<12} {'Mean ESS':<12} {'Mean τ':<12} {'Efficiency':<12}")
print("-" * 80)

for method, acc_rate, eff, autocorr in zip(methods, all_acceptance, all_eff, all_autocorr):
    efficiency = np.mean(eff) / (min_samples - burnin_unified)
    print(f"{method:<20} {acc_rate:<12.3f} {np.mean(eff):<12.1f} {np.mean(autocorr):<12.2f} {efficiency:<12.3f}")

# 相対性能
print(f"\n=== 相対性能（Random Walk MH基準） ===")
base_eff = np.mean(rwmh_eff)
base_tau = np.mean(rwmh_autocorr)

for method, eff, autocorr in zip(methods, all_eff, all_autocorr):
    eff_ratio = np.mean(eff) / base_eff
    tau_ratio = base_tau / np.mean(autocorr)  # 小さいほうが良いので逆数
    print(f"{method}: ESS ratio = {eff_ratio:.2f}, τ improvement = {tau_ratio:.2f}x")

## 6.5 実用的なMCMCライブラリの紹介

実際の研究・開発では、専用ライブラリを使用することが一般的です。主要なライブラリを紹介します。

In [ ]:
# 主要なMCMCライブラリの概要と使用例
print("=== 主要なMCMCライブラリ ===")
print()

libraries_info = {
    "PyMC": {
        "description": "Pythonで最も人気のあるベイズ統計ライブラリ",
        "features": [
            "直感的なモデル記述",
            "自動微分によるHMC/NUTS",
            "豊富な分布ライブラリ",
            "変分推論サポート",
            "優れた可視化機能"
        ],
        "use_cases": "階層モデル、時系列分析、機械学習"
    },
    "Stan (PyStan)": {
        "description": "高性能なベイズ推論プラットフォーム",
        "features": [
            "専用言語による高速実行",
            "最先端のNUTS実装",
            "自動微分と最適化",
            "R, Python, Julia等の多言語サポート",
            "詳細な診断機能"
        ],
        "use_cases": "複雑なモデル、大規模データ、研究用途"
    },
    "TensorFlow Probability": {
        "description": "TensorFlowベースの確率的プログラミング",
        "features": [
            "GPU加速サポート",
            "深層学習との統合",
            "変分推論と正規化フロー",
            "大規模データ対応",
            "分散計算サポート"
        ],
        "use_cases": "ベイズ深層学習、大規模推論"
    },
    "PyTorch/Pyro": {
        "description": "PyTorchベースの確率的プログラミング",
        "features": [
            "動的計算グラフ",
            "HMC/NUTSサポート",
            "変分推論",
            "ガイド付き推論",
            "研究向け柔軟性"
        ],
        "use_cases": "研究開発、カスタムモデル"
    },
    "emcee": {
        "description": "アンサンブルサンプラー専門ライブラリ",
        "features": [
            "Affine Invariant Ensemble Sampler",
            "高次元問題に効果的",
            "並列化サポート",
            "シンプルなAPI",
            "天体物理学で人気"
        ],
        "use_cases": "高次元パラメータ推定、物理学応用"
    }
}

for lib_name, info in libraries_info.items():
    print(f"【{lib_name}】")
    print(f"概要: {info['description']}")
    print("主な機能:")
    for feature in info['features']:
        print(f"  • {feature}")
    print(f"適用分野: {info['use_cases']}")
    print()

# ライブラリ選択の指針
print("=== ライブラリ選択の指針 ===")
print()

selection_guide = {
    "初学者・一般的な用途": "PyMC - 直感的で豊富なドキュメント",
    "高性能・複雑なモデル": "Stan - 最適化された実装と豊富な診断",
    "深層学習との統合": "TensorFlow Probability, Pyro - GPU活用と大規模データ",
    "研究・カスタム実装": "Pyro, 自作実装 - 柔軟性と制御",
    "高次元パラメータ推定": "emcee - 効率的なアンサンブル手法",
    "教育・理解": "自作実装 - アルゴリズムの詳細理解"
}

for use_case, recommendation in selection_guide.items():
    print(f"{use_case}: {recommendation}")

print()
print("=== パフォーマンス比較の目安 ===")
print()

performance_comparison = {
    "実行速度": "Stan > TFP ≈ Pyro > PyMC > emcee > 自作実装",
    "学習コスト": "PyMC < emcee < TFP ≈ Pyro < Stan < 自作実装",
    "柔軟性": "自作実装 > Pyro ≈ TFP > Stan ≈ PyMC > emcee",
    "診断機能": "Stan > PyMC > TFP ≈ Pyro > emcee > 自作実装",
    "コミュニティ": "PyMC ≈ Stan > TFP > Pyro > emcee > 自作実装"
}

for aspect, ranking in performance_comparison.items():
    print(f"{aspect}: {ranking}")

# 実装例のテンプレート（疑似コード）
print()
print("=== 各ライブラリの実装例（疑似コード） ===")
print()

print("【PyMC例】")
pymc_example = """
import pymc as pm

with pm.Model() as model:
    # 事前分布
    beta = pm.Normal('beta', mu=0, sigma=10, shape=p)
    sigma = pm.HalfNormal('sigma', sigma=1)
    
    # 尤度
    mu = pm.math.dot(X, beta)
    y_obs = pm.Normal('y_obs', mu=mu, sigma=sigma, observed=y)
    
    # サンプリング
    trace = pm.sample(2000, tune=1000, cores=4)
"""
print(pymc_example)

print("【Stan例】")
stan_example = """
# model.stan
data {
  int<lower=0> n;
  int<lower=0> p;
  matrix[n, p] X;
  vector[n] y;
}
parameters {
  vector[p] beta;
  real<lower=0> sigma;
}
model {
  beta ~ normal(0, 10);
  sigma ~ normal(0, 1);
  y ~ normal(X * beta, sigma);
}

# Python
import stan
posterior = stan.build(stan_code, data=data)
fit = posterior.sample(num_chains=4, num_samples=2000)
"""
print(stan_example)

## 6.6 演習問題

### 問題1：HMCのパラメータ調整
異なるステップサイズとリープフロッグステップ数でHMCの性能を比較してください。

In [ ]:
# 問題1: HMCパラメータの最適化
def hmc_parameter_tuning(log_prob_fn, grad_log_prob_fn, initial_q, n_samples=2000):
    """
    HMCのパラメータ（ε, L）の最適化実験
    
    ヒント：
    1. 異なる(ε, L)の組み合わせでHMCを実行
    2. 受理率、有効サンプルサイズ、ハミルトニアン誤差を記録
    3. 最適なパラメータ組み合わせを特定
    4. トレードオフ関係を可視化
    """
    # ここに実装してください
    epsilons = [0.05, 0.1, 0.2, 0.3, 0.5]
    L_values = [5, 10, 20, 30, 50]
    
    results = {}
    
    # パラメータグリッドサーチの実装
    # ...
    
    pass  # 学習者が実装

# テスト実行
# results = hmc_parameter_tuning(
#     lambda x: multivariate_normal_log_prob(x, mu_target, cov_target),
#     lambda x: multivariate_normal_grad_log_prob(x, mu_target, cov_target),
#     np.array([0.0, 0.0])
# )

### 問題2：並列MCMC
複数のチェーンを並列実行し、結果を統合する手法を実装してください。

In [ ]:
# 問題2: 並列MCMCの実装
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp

def parallel_mcmc_chains(sampler_func, sampler_args, n_chains=4, n_samples=2000):
    """
    複数チェーンの並列実行
    
    Parameters:
    - sampler_func: サンプリング関数
    - sampler_args: サンプラー引数のリスト（チェーンごと）
    - n_chains: チェーン数
    - n_samples: チェーンあたりのサンプル数
    
    実装のヒント：
    1. ProcessPoolExecutorを使用
    2. 異なる初期値・乱数シードでチェーンを開始
    3. Gelman-Rubin診断で収束確認
    4. チェーンの結合と統計計算
    """
    # ここに実装してください
    pass  # 学習者が実装

def single_chain_wrapper(args):
    """
    単一チェーン実行のラッパー関数
    """
    # 並列処理用のラッパー実装
    pass  # 学習者が実装

# 使用例（疑似コード）
# chains_results = parallel_mcmc_chains(
#     sampler_func=random_walk_mh,
#     sampler_args=[
#         (log_prob_fn, initial_values[i], n_samples, step_size) 
#         for i in range(n_chains)
#     ]
# )

## まとめ

この章では、高度なMCMC手法について学習しました：

### 学習した手法

1. **ハミルトニアンモンテカルロ法（HMC）**：
   - 物理学的原理の活用
   - 勾配情報による効率的探索
   - リープフロッグ積分による数値計算
   - 高次元問題での優位性

2. **No-U-Turn Sampler（NUTS）**：
   - HMCの自動パラメータ調整
   - U-turn検出による最適軌跡長
   - 適応的ステップサイズ調整
   - 実用性の向上

3. **適応的MCMC**：
   - 実行中のパラメータ自動調整
   - 経験共分散による提案分布最適化
   - ユーザー介入の削減

### 性能比較の結果

| 手法 | 受理率 | 効率 | 適用範囲 | 実装難易度 |
|------|--------|------|----------|------------|
| Random Walk MH | 中 | 低 | 汎用 | 易 |
| HMC | 高 | 高 | 連続・微分可能 | 中 |
| NUTS | 高 | 最高 | 連続・微分可能 | 難 |
| Adaptive MH | 中〜高 | 中〜高 | 汎用 | 中 |

### 実用的な指針

**手法選択の基準**：
- **勾配利用可能** → HMC/NUTS
- **高次元問題** → HMC/NUTS/Ensemble methods
- **離散パラメータ** → Gibbs/MH
- **計算資源制約** → Adaptive MH
- **実装簡単さ重視** → Random Walk MH

**ライブラリ活用**：
- 研究・開発では専用ライブラリの使用を強く推奨
- PyMC, Stan, TensorFlow Probabilityが主要選択肢
- アルゴリズム理解のための自作実装も有効

### 今後の発展

- **変分推論**：近似的だが高速なベイズ推論
- **正規化フロー**：複雑な事後分布の表現
- **GPU加速**：大規模データへの対応
- **自動微分**：勾配計算の自動化
- **ベイズ深層学習**：ニューラルネットワークとの統合

MCMCは現代の統計学・機械学習における中核技術として、今後も発展を続けることが予想されます。基礎理論の理解と実践的な応用スキルの両方を身につけることが重要です。